## Homework: Vector Search

In this homework, I'll experiment with vector with and without Elasticsearch

## Q1. Getting the embeddings model

First, I will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)


In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

c:\Users\marti.MARTIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Create the embedding for this user question:

In [2]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]

0.07822262

What's the first value of the resulting vector?

* -0.24
* -0.04
* **0.07**
* 0.27

## Prepare the documents

Now I will create the embeddings for the documents.

I load the documents with ids that have been prepared in the module:

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [4]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'id': '1f6520ca'}

In [5]:
subset_docs = [d for d in documents if d['course'] == 'machine-learning-zoomcamp']
len(subset_docs)

375

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 

In [6]:
import numpy as np
from tqdm.auto import tqdm

In [7]:
embeddings = []
for doc in tqdm(subset_docs):
    qa_text = '{question} {text}'.format(**doc)
    qa_text_vector = embedding_model.encode(qa_text)
    embeddings.append(qa_text_vector)

100%|██████████| 375/375 [00:22<00:00, 16.38it/s]


In [8]:
X = np.array(embeddings)
print(X)
print(X.shape)

[[ 0.08805911  0.01559365  0.07925579 ...  0.00745124  0.00241913
   0.01147132]
 [ 0.07066914 -0.01930562  0.0773837  ...  0.04495562  0.01078554
  -0.02316591]
 [ 0.10229155 -0.0166324   0.03414484 ...  0.03215319 -0.04317445
  -0.04574377]
 ...
 [-0.00768201  0.01075665  0.01190489 ...  0.06066166 -0.03393208
   0.01605328]
 [ 0.13408284 -0.03755201  0.0197653  ...  0.02163629 -0.01921146
   0.03690196]
 [ 0.04386011 -0.0130077   0.06373127 ...  0.03338454  0.02258543
  -0.0630587 ]]
(375, 768)


## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return 1.0). This means that in order
to compute the cosine similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


In [10]:
v = embedding_model.encode(user_question)

In [11]:
len(v)

768

In [12]:
v.dot(v)

0.9999999

In [13]:
scores = X.dot(v)
scores.max()

0.65065753

In [14]:
scores.argmax()

14

In [15]:
subset_docs[14]

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'id': 'ee58a693'}

What's the highest score in the results?

- 65.0 
- 6.5
- **0.65**
- 0.065

## Vector search
We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [18]:
(-scores).argsort()

array([ 14,  17,   5, 371,  20,   6,   1,  13,  15,  18,  19,  22,   9,
       329,   4, 140,  11,   0,   7, 130, 374,  61, 333,   2,   8,  87,
        62,  50,  16, 323, 362, 152,  21, 162, 153,  10, 324, 363, 334,
       204, 330, 336, 122, 361, 327, 134, 305, 372,   3, 231, 342, 233,
       217, 185, 214, 317, 191, 321,  67, 236, 253, 194, 189, 340, 263,
       220,  66, 219, 232, 279,  27, 344, 206, 222, 358, 155, 190, 196,
       246, 331, 176, 159, 156, 247, 313, 262, 322,  26, 197, 210, 105,
       302, 300,  23, 280,  86, 291, 168,  93, 304,  80, 182, 127, 369,
       250, 137, 293, 202, 347, 199, 188,  97, 373, 151, 227, 359, 249,
       283, 208, 240, 235,  69, 172, 337, 111, 242,  88, 271, 292, 264,
       109, 121, 239,  89, 269, 160, 287, 338,  46, 131, 351,  44, 157,
       356,  32,  68, 244, 128, 174, 310, 177, 101, 350, 164, 237, 133,
       180, 314, 238,  31, 272,  83, 307,  38,  35, 328, 203, 303, 252,
       367, 297,  63, 298,  65, 316, 311, 368, 319,  30, 106,  7

In [17]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=subset_docs, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine
Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

First, load the ground truth dataset:

In [19]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0:5]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Does this course have a GitHub repository for the sign-up link?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'How can I structure my questions and answers for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5.

In [20]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [21]:
def evaluate(ground_truth, search_engine):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        question = q['question']
        v_q = embedding_model.encode(question)
        results = search_engine.search(v_q, num_results=5)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [22]:
evaluate(ground_truth, search_engine)

100%|██████████| 1830/1830 [00:54<00:00, 33.63it/s]


{'hit_rate': 0.9398907103825137}

What did you get?

+ **0.93**
+ 0.73
+ 0.53
+ 0.33

## Q5. Indexing with Elasticsearch
Now let's index these documents with elasticsearch

+ Create the index with the same settings as in the module (but change the dimensions)
+ Index the embeddings (note: you've already computed them)

In [24]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qa_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
     }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [25]:
for i, doc in tqdm(enumerate(subset_docs)):
    doc['qa_text_vector'] = X[i].tolist()
    es_client.index(index=index_name, document=doc)

375it [00:25, 14.93it/s]


After indexing, let's perform the search of the same query from Q1.

In [26]:
def elastic_search_knn(vector):
    knn = {
        "field": 'qa_text_vector',
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append({
        "source": hit['_source'],
        "score": hit['_score']
    })

    return result_docs

In [27]:
elastic_search_knn(v)

[{'source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'},
  'score': 0.82532895},
 {'source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s

What's the ID of the document with the highest score?

'id': 'ee58a693'

## Q6. Hit-rate for Elasticsearch
The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

In [28]:
def evaluate(ground_truth, elastic_search_knn):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        question = q['question']
        v_q = embedding_model.encode(question)
        results = elastic_search_knn(v_q)
        relevance = [d['source']['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [29]:
evaluate(ground_truth, elastic_search_knn)

100%|██████████| 1830/1830 [02:42<00:00, 11.24it/s]


{'hit_rate': 0.9398907103825137}

What's hitrate for our dataset for Elastic?

+ **0.93**
+ 0.73
+ 0.53
+ 0.33